# 1.Package Importing



In [ ]:
! pip install pyspark
! wget https://setup.johnsnowlabs.com/colab.sh -O - | bash
! wget https://setup.johnsnowlabs.com/colab.sh -O - | bash /dev/stdin -p 3.2.1 -s 4.2.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 53.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=9780bdc685d557638693606f3b7204fa2277acbf568ed3ceb772cd0009b0a0d7
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2022-12-14 13:40:00--  https://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-12-14 13:40:01--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/m

In [ ]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
import os

spark = sparknlp.start(gpu = True)# for GPU training >> sparknlp.start(gpu = True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.4
Apache Spark version: 3.2.1


In [ ]:
import pandas as pd
#create spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark=SparkSession.builder.appName('nlp').getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2.Data preprocessing

In [ ]:
#text_df= spark.read.csv("/content/drive/MyDrive/cs631_final_project/data/kindle_reviews.csv",inferSchema=True,header=True,sep=',')
text_df= spark.read.csv("/content/drive/MyDrive/cs631/kindle_reviews.csv",inferSchema=True,header=True,sep=',')
df_raw = text_df.select('overall','reviewText')


In [ ]:
df_raw.printSchema()

root
 |-- overall: integer (nullable = true)
 |-- reviewText: string (nullable = true)



In [ ]:
from pyspark.sql.functions import rand
df_raw.orderBy(rand()).show(10,False)

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
df = df_raw.withColumn("Label", text_df.overall.cast('float')).drop('overall')
df = df.dropna(how='any')
df.groupBy('label').count().show()
df.count()

+-----+------+
|label| count|
+-----+------+
|  5.0|575246|
|  2.0| 34130|
|  3.0| 96193|
|  1.0| 23018|
|  4.0|254010|
+-----+------+



982597

In [ ]:
df.printSchema()

root
 |-- reviewText: string (nullable = true)
 |-- Label: float (nullable = true)



In [ ]:
# convert labels to sentiments
df = df.withColumn("label", when(df.Label >= 3.999,1) \
#      .when(df.Label <= 3.001,0) \
      .otherwise(0))
df.show()

+--------------------+-----+
|          reviewText|label|
+--------------------+-----+
|I enjoy vintage b...|    1|
|This book is a re...|    1|
|This was a fairly...|    1|
|I'd never read an...|    1|
|If you like perio...|    1|
|A beautiful in-de...|    1|
|I enjoyed this on...|    1|
|Never heard of Am...|    1|
|Darth Maul workin...|    1|
|This is a short s...|    1|
|I think I have th...|    1|
|Title has nothing...|    1|
|Well written. Int...|    0|
|Troy Denning's no...|    0|
|I am not for sure...|    1|
|I really enjoyed ...|    1|
|Great read enjoye...|    1|
|Another well writ...|    0|
|This one promises...|    1|
|"I have a version...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
# Add length to the dataframe
from pyspark.sql.functions import length
df= df.withColumn('length',length(df['reviewText']))
df.show()

+--------------------+-----+------+
|          reviewText|label|length|
+--------------------+-----+------+
|I enjoy vintage b...|    1|   294|
|This book is a re...|    0|   455|
|This was a fairly...|    0|   375|
|I'd never read an...|    1|   101|
|If you like perio...|    0|   130|
|A beautiful in-de...|    0|   163|
|I enjoyed this on...|    0|   163|
|Never heard of Am...|    0|   286|
|Darth Maul workin...|    1|   127|
|This is a short s...|    0|   466|
|I think I have th...|    1|    93|
|Title has nothing...|    0|   255|
|Well written. Int...|    0|   181|
|Troy Denning's no...|    0|  2295|
|I am not for sure...|    1|   266|
|I really enjoyed ...|    1|   108|
|Great read enjoye...|    1|   112|
|Another well writ...|    0|   274|
|This one promises...|    1|   115|
|"I have a version...|    0|    82|
+--------------------+-----+------+
only showing top 20 rows



In [ ]:
import string, re
# remove punctuation
rdd = df.rdd.map(lambda x: (x[1],x[2],re.sub(r'[^\w\s]', ' ', x[0])))
# lowercase
rdd = rdd.map(lambda x: (x[0],x[1],x[2].lower()))
# remove numbers
rdd = rdd.map(lambda x: (x[0],x[1],re.sub(r'\d+', '', x[2])))
# remove double spaces
rdd = rdd.map(lambda x: (x[0],x[1]," ".join(x[2].split())))
# only keep letters
rdd = rdd.map(lambda x: (x[0],x[1],re.sub('[^a-zA-Z]+', ' ', x[2])))

new_df = rdd.toDF(["Label","length","reviewText"])
new_df.show(5)

+-----+------+--------------------+
|Label|length|          reviewText|
+-----+------+--------------------+
|    1|   294|i enjoy vintage b...|
|    0|   455|this book is a re...|
|    0|   375|this was a fairly...|
|    1|   101|i d never read an...|
|    0|   130|if you like perio...|
+-----+------+--------------------+
only showing top 5 rows



In [ ]:
#Tokenize
from pyspark.ml.feature import RegexTokenizer
tokenization = RegexTokenizer(inputCol='reviewText',outputCol='tokens', pattern = "\\W")
tokenized_df= tokenization.transform(df)
tokenized_df.show()

+--------------------+-----+------+--------------------+
|          reviewText|label|length|              tokens|
+--------------------+-----+------+--------------------+
|I enjoy vintage b...|    1|   294|[i, enjoy, vintag...|
|This book is a re...|    0|   455|[this, book, is, ...|
|This was a fairly...|    0|   375|[this, was, a, fa...|
|I'd never read an...|    1|   101|[i, d, never, rea...|
|If you like perio...|    0|   130|[if, you, like, p...|
|A beautiful in-de...|    0|   163|[a, beautiful, in...|
|I enjoyed this on...|    0|   163|[i, enjoyed, this...|
|Never heard of Am...|    0|   286|[never, heard, of...|
|Darth Maul workin...|    1|   127|[darth, maul, wor...|
|This is a short s...|    0|   466|[this, is, a, sho...|
|I think I have th...|    1|    93|[i, think, i, hav...|
|Title has nothing...|    0|   255|[title, has, noth...|
|Well written. Int...|    0|   181|[well, written, i...|
|Troy Denning's no...|    0|  2295|[troy, denning, s...|
|I am not for sure...|    1|   

In [ ]:
# Stopwords Removal
from pyspark.ml.feature import StopWordsRemover
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')
df=stopword_removal.transform(tokenized_df)
df.show()

+--------------------+-----+------+--------------------+--------------------+
|          reviewText|label|length|              tokens|      refined_tokens|
+--------------------+-----+------+--------------------+--------------------+
|I enjoy vintage b...|    1|   294|[i, enjoy, vintag...|[enjoy, vintage, ...|
|This book is a re...|    0|   455|[this, book, is, ...|[book, reissue, o...|
|This was a fairly...|    0|   375|[this, was, a, fa...|[fairly, interest...|
|I'd never read an...|    1|   101|[i, d, never, rea...|[d, never, read, ...|
|If you like perio...|    0|   130|[if, you, like, p...|[like, period, pi...|
|A beautiful in-de...|    0|   163|[a, beautiful, in...|[beautiful, depth...|
|I enjoyed this on...|    0|   163|[i, enjoyed, this...|[enjoyed, one, th...|
|Never heard of Am...|    0|   286|[never, heard, of...|[never, heard, am...|
|Darth Maul workin...|    1|   127|[darth, maul, wor...|[darth, maul, wor...|
|This is a short s...|    0|   466|[this, is, a, sho...|[short, 

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *
from pyspark.ml.feature import *

In [ ]:
len_udf = udf(lambda s: len(s), IntegerType())
refined_text_df = df.withColumn("token_count", len_udf(col('refined_tokens')))
refined_text_df.show()

+--------------------+-----+------+--------------------+--------------------+-----------+
|          reviewText|label|length|              tokens|      refined_tokens|token_count|
+--------------------+-----+------+--------------------+--------------------+-----------+
|I enjoy vintage b...|    1|   294|[i, enjoy, vintag...|[enjoy, vintage, ...|         29|
|This book is a re...|    0|   455|[this, book, is, ...|[book, reissue, o...|         50|
|This was a fairly...|    0|   375|[this, was, a, fa...|[fairly, interest...|         32|
|I'd never read an...|    1|   101|[i, d, never, rea...|[d, never, read, ...|          9|
|If you like perio...|    0|   130|[if, you, like, p...|[like, period, pi...|         13|
|A beautiful in-de...|    0|   163|[a, beautiful, in...|[beautiful, depth...|         19|
|I enjoyed this on...|    0|   163|[i, enjoyed, this...|[enjoyed, one, th...|         16|
|Never heard of Am...|    0|   286|[never, heard, of...|[never, heard, am...|         32|
|Darth Mau

In [ ]:
#refined_text_df.orderBy("token_count").show(10)

In [ ]:
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features').setVocabSize(10000)

In [ ]:
%%time
cv_text_df= count_vec.fit(refined_text_df).transform(refined_text_df)


CPU times: user 450 ms, sys: 51.3 ms, total: 502 ms
Wall time: 1min 22s


In [ ]:
cv_text_df = cv_text_df.select(['refined_tokens','token_count','features','Label'])
cv_text_df.show(10)

+--------------------+-----------+--------------------+-----+
|      refined_tokens|token_count|            features|Label|
+--------------------+-----------+--------------------+-----+
|[enjoy, vintage, ...|         29|(10000,[0,2,7,14,...|    1|
|[book, reissue, o...|         50|(10000,[0,3,5,7,1...|    0|
|[fairly, interest...|         32|(10000,[1,2,5,13,...|    0|
|[d, never, read, ...|          9|(10000,[2,3,6,50,...|    1|
|[like, period, pi...|         13|(10000,[5,12,21,1...|    0|
|[beautiful, depth...|         19|(10000,[5,46,111,...|    0|
|[enjoyed, one, th...|         16|(10000,[3,8,11,20...|    0|
|[never, heard, am...|         32|(10000,[0,1,3,5,2...|    0|
|[darth, maul, wor...|         14|(10000,[1,10,16,4...|    1|
|[short, story, fo...|         45|(10000,[1,3,6,8,1...|    0|
+--------------------+-----------+--------------------+-----+
only showing top 10 rows



In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
df_assembler = VectorAssembler(inputCols=['features','token_count'],outputCol='features_vec')
model_text_df = df_assembler.transform(cv_text_df)

# 3.Model Training

In [ ]:
#convert labels to sentiments
# df_sent = df.withColumn("label", when(df.label >= 3.999,1) \
# #      .when(df.Label <= 3.001,0) \
#       .otherwise(0))
df_sent = df.withColumn('label', col('label').cast('string'))
training_df,testdf= df_sent.randomSplit([0.75,0.25])
training_df.count()

## 3.1 Logistic Regression


In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
training_df.printSchema()
#training_df.groupBy('label').count().show()

root
 |-- reviewText: string (nullable = true)
 |-- label: string (nullable = false)



In [ ]:
%%time
log_model =LogisticRegression(featuresCol='features_vec',labelCol='Label').fit(training_df)

In [ ]:
log_pred = log_model.evaluate(test_df).predictions

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
predictions = log_model.transform(test_df)
predictions.count()

In [ ]:
# Model Evaluation
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
# evaluator.evaluate(predictions)

In [ ]:
accuracy = predictions[(predictions.Label == predictions.prediction)].count()
float(accuracy)/predictions.count()

0.6256390090444357

## 3.2 Naive Bayes


In [ ]:
from pyspark.ml.classification import NaiveBayes

In [ ]:
%%time
nb_model = NaiveBayes(featuresCol='features_vec',labelCol='Label', smoothing=1).fit(training_df)

CPU times: user 37.8 ms, sys: 4.5 ms, total: 42.3 ms
Wall time: 3.91 s


In [ ]:
nb_pred = nb_model.transform(test_df)
nb_pred.schema

StructType(List(StructField(refined_tokens,ArrayType(StringType,true),true),StructField(token_count,IntegerType,true),StructField(features,VectorUDT,true),StructField(Label,IntegerType,false),StructField(features_vec,VectorUDT,true),StructField(rawPrediction,VectorUDT,true),StructField(probability,VectorUDT,true),StructField(prediction,DoubleType,false)))

In [ ]:
acc_nb = nb_pred[(nb_pred.Label == nb_pred.prediction)].count()
float(acc_nb)/nb_pred.count()

0.7235548564687377

In [ ]:
nb_pred.select('Label').show(5)

+-----+
|Label|
+-----+
|    1|
|    1|
|    1|
|    0|
|    0|
+-----+
only showing top 5 rows



In [ ]:
nb_pred.select('prediction').show(5)

+----------+
|prediction|
+----------+
|       1.0|
|       0.0|
|       1.0|
|       0.0|
|       0.0|
+----------+
only showing top 5 rows



## 3.3 Random Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
%%time
rf_model = RandomForestClassifier(labelCol="Label", \
                            featuresCol="features_vec", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32).fit(training_df)

CPU times: user 173 ms, sys: 21.9 ms, total: 195 ms
Wall time: 25.6 s


In [ ]:
rf_pred = rf_model.transform(test_df)
rf_pred.count()

2543

In [ ]:
acc_rf = rf_pred[(rf_pred.Label == rf_pred.prediction)].count()
float(acc_rf)/rf_pred.count()

0.5505308690523004

# 4.ClassiferDL with Word Embeddings and Text Preprocessing

In [ ]:
# df = df.dropna(how='any')
# #df = new_df
# #convert labels to sentiments
# df_sent = df.withColumn("Label", when(df.Label >= 3.999,1) \
# #      .when(df.Label <= 3.001,0) \
#       .otherwise(0))
# df_sent.show()
df_sent = df.withColumn('Label', col('Label').cast('string'))
traindf,testdf= df_sent.randomSplit([0.75,0.25])
traindf.count()

7572

In [ ]:
traindf.show(5)

+--------------------+-----+
|          reviewText|Label|
+--------------------+-----+
|!!!!??? (sorry co...|    1|
|"""*69"" starts w...|    0|
|"""...the world i...|    0|
|"""A Dundee Chris...|    1|
|"""A Hard Man is ...|    0|
+--------------------+-----+
only showing top 5 rows



In [ ]:
document_assembler = DocumentAssembler() \
            .setInputCol("reviewText") \
            .setOutputCol("document")
    
tokenizer = Tokenizer() \
            .setInputCols("document") \
            .setOutputCol("token")
          
normalizer = Normalizer() \
            .setInputCols("token") \
            .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
            .setInputCols("normalized")\
            .setOutputCol("cleanTokens")\
            .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
            .setInputCols(["cleanTokens"]) \
            .setOutputCol("lemma")


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


## 4.1 Glove 100d embeddings

In [ ]:
glove_embeddings = WordEmbeddingsModel().pretrained() \
                        .setInputCols(["document",'lemma'])\
                        .setOutputCol("embeddings")\
                        .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
                        .setInputCols(["document", "embeddings"]) \
                        .setOutputCol("sentence_embeddings") \
                        .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
                        .setInputCols(["sentence_embeddings"])\
                        .setOutputCol("class")\
                        .setLabelColumn("Label")\
                        .setMaxEpochs(3)\
                        .setEnableOutputLogs(True)
                        #.setOutputLogsPath('logs')

clf_pipeline = Pipeline(
    stages=[
        document_assembler, 
        tokenizer,
        normalizer,
        stopwords_cleaner, 
        lemma, 
        glove_embeddings,
        embeddingsSentence,
        classsifierdl])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [ ]:
traindf.show(5)

+--------------------+-----+
|          reviewText|Label|
+--------------------+-----+
|!!!!??? (sorry co...|    1|
|"""*69"" starts w...|    0|
|"""...the world i...|    0|
|"""A Dundee Chris...|    1|
|"""A Hard Man is ...|    0|
+--------------------+-----+
only showing top 5 rows



In [ ]:
traindf.count()

736280

In [ ]:
traindf.printSchema()

root
 |-- reviewText: string (nullable = true)
 |-- Label: string (nullable = false)



In [ ]:
%%time
clf_pipelineModel = clf_pipeline.fit(traindf)

In [ ]:
import os
log_file_name = os.listdir("/root/annotator_logs")[0]

with open("/root/annotator_logs/"+log_file_name, "r") as log_file :
    print(log_file.read())

In [ ]:
# get the predictions on test Set
preds = clf_pipelineModel.transform(testdf)

In [ ]:
preds.select('Label','reviewText',"class.result")

DataFrame[Label: string, reviewText: string, result: array<string>]

In [ ]:
preds_df = preds.select('Label','reviewText',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [ ]:
preds_df

In [ ]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['Label'], preds_df['result']))

## 4.2 Bert Embeddings

In [ ]:
document_assembler = DocumentAssembler() \
                .setInputCol("reviewText") \
                .setOutputCol("document")

tokenizer = Tokenizer() \
                .setInputCols(["document"]) \
                .setOutputCol("token")
      
bert_embeddings = BertEmbeddings().pretrained(name='small_bert_L4_256', lang='en') \
                .setInputCols(["document",'token'])\
                .setOutputCol("embeddings")

embeddingsSentence = SentenceEmbeddings() \
                .setInputCols(["document", "embeddings"]) \
                .setOutputCol("sentence_embeddings") \
                .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
                .setInputCols(["sentence_embeddings"])\
                .setOutputCol("class")\
                .setLabelColumn("Label")\
                .setMaxEpochs(5)\
                .setLr(0.001)\
                .setBatchSize(16)\
                .setEnableOutputLogs(True)
                #.setOutputLogsPath('logs')

bert_clf_pipeline = Pipeline(stages=[document_assembler,
                                     tokenizer,
                                     bert_embeddings,
                                     embeddingsSentence,
                                     classsifierdl])

small_bert_L4_256 download started this may take some time.
Approximate size to download 40.5 MB
[OK!]


In [ ]:
! rm -r /root/annotator_logs

In [ ]:
# training will take some time due to Bert (use GPU runtime when possible)

%%time

bert_clf_pipelineModel = bert_clf_pipeline.fit(traindf)

In [ ]:
log_files = os.listdir("/root/annotator_logs")
log_files

['ClassifierDLApproach_d4a81f029240.log']

In [ ]:
log_file_name = os.listdir("/root/annotator_logs")[0]

with open("/root/annotator_logs/"+log_file_name, "r") as log_file :
    print(log_file.read())

In [ ]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

preds_bert = bert_clf_pipelineModel.transform(testdf)

preds_bert_df = preds_bert.select('Label','reviewText',"class.result").toPandas()

preds_bert_df['result'] = preds_bert_df['result'].apply(lambda x : x[0])

print (classification_report(preds_bert_df['Label'], preds_bert_df['result']))


              precision    recall  f1-score   support

           0       0.65      0.41      0.50       575
           1       0.84      0.93      0.88      1853

    accuracy                           0.81      2428
   macro avg       0.74      0.67      0.69      2428
weighted avg       0.79      0.81      0.79      2428

